In [ ]:
from pymongo import MongoClient
import requests
from bs4 import BeautifulSoup
import logging

# Konfiguracja logowania
logging.basicConfig(level=logging.INFO, format='%(asctime)s - %(levelname)s - %(message)s')

# Połączenie z MongoDB
client = MongoClient()
db = client['czech_liga_pro_test']
matches_collection = db['matches']

# Krok 1: Znalezienie duplikatów
query = { '$expr': { '$eq': ['$home_name', '$away_name'] } }
duplicates = list(matches_collection.find(query).sort("datetime", 1))
logging.info(f"Znaleziono {len(duplicates)} rekordów z duplikatami imion.")

if not duplicates:
    logging.info("Brak rekordów do aktualizacji.")
    exit()

# Funkcja do pobrania prawidłowych nazw z tt_url
def fetch_correct_names(tt_url):
    """
    Funkcja pobiera z danej strony (tt_url) imię i nazwisko zawodnika "home" 
    oraz imię i nazwisko zawodnika "away" na podstawie kodu HTML.
    """
    try:
        response = requests.get(tt_url)
        response.raise_for_status()  # Jeśli status != 200 rzuca wyjątek
        soup = BeautifulSoup(response.text, 'html.parser')
        
        # Znajdujemy wszystkie divy o klasie "col-4 player"
        players_divs = soup.find_all("div", class_="col-4 player")
        
        # Pierwszy div - home, drugi div - away
        home_name = players_divs[0].find("a").get_text(strip=True)
        away_name = players_divs[1].find("a").get_text(strip=True)

        return home_name, away_name
    except Exception as e:
        logging.error(f"Błąd podczas pobierania danych z {tt_url}: {e}")
        return None, None

# Krok 2-4: Przetwarzanie każdego duplikatu
for doc in duplicates:
    tt_url = doc.get('tt_url')
    if not tt_url:
        logging.warning(f"Brak tt_url dla dokumentu _id: {doc['_id']}")
        continue
    
    logging.info(f"Przetwarzanie dokumentu _id: {doc['_id']} z tt_url: {tt_url}")
    
    # Krok 2: Pobranie prawidłowych nazw
    home_name_correct, away_name_correct = fetch_correct_names(tt_url)
    
    if not home_name_correct or not away_name_correct:
        logging.warning(f"Nie udało się pobrać prawidłowych nazw dla _id: {doc['_id']}")
        continue
    
    # Sprawdzenie, czy nazwy faktycznie się różnią
    if home_name_correct != doc['home_name'] or away_name_correct != doc['away_name']:
        # Krok 4: Aktualizacja dokumentu
        update_result = matches_collection.update_one(
            { '_id': doc['_id'] },
            { '$set': { 'home_name': home_name_correct, 'away_name': away_name_correct } }
        )
        if update_result.modified_count == 1:
            logging.info(f"Zaktualizowano dokument _id: {doc['_id']}")
        else:
            logging.warning(f"Nie udało się zaktualizować dokumentu _id: {doc['_id']}")
    else:
        logging.info(f"Brak zmian dla dokumentu _id: {doc['_id']}")

logging.info("Proces aktualizacji zakończony.")


2024-12-22 13:25:35,626 - INFO - Znaleziono 46 rekordów z duplikatami imion.
2024-12-22 13:25:35,628 - INFO - Przetwarzanie dokumentu _id: 675bebd80dfd8a968dc08886 z tt_url: https://tt.league-pro.com/games/117364
2024-12-22 13:25:36,646 - INFO - Zaktualizowano dokument _id: 675bebd80dfd8a968dc08886
2024-12-22 13:25:36,648 - INFO - Przetwarzanie dokumentu _id: 675be6f40dfd8a968dc08488 z tt_url: https://tt.league-pro.com/games/118389
2024-12-22 13:25:37,793 - INFO - Zaktualizowano dokument _id: 675be6f40dfd8a968dc08488
2024-12-22 13:25:37,799 - INFO - Przetwarzanie dokumentu _id: 675be68c0dfd8a968dc08443 z tt_url: https://tt.league-pro.com/games/118589
2024-12-22 13:25:38,760 - INFO - Zaktualizowano dokument _id: 675be68c0dfd8a968dc08443
2024-12-22 13:25:38,762 - INFO - Przetwarzanie dokumentu _id: 675be68e0dfd8a968dc08444 z tt_url: https://tt.league-pro.com/games/118591
2024-12-22 13:25:40,012 - INFO - Zaktualizowano dokument _id: 675be68e0dfd8a968dc08444
2024-12-22 13:25:40,014 - INFO 